## Memory usage and FLOPs of each model

In [1]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import time
from tqdm.notebook import tqdm
from transformers import pipeline
import torch
import psutil
import subprocess
import gc
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from thop import profile, clever_format
%env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [2]:
print(torch.cuda.is_available())
print(torch.cuda.current_device())

if torch.cuda.is_available():
    device = 0  # Use the first GPU
else:
    device = -1  # Use the CPU

True
0


## Functions to monitor memory usage

In [2]:
# Function to get current CPU memory usage
def get_cpu_memory_usage():
    process = psutil.Process()
    mem_info = process.memory_info()
    return int(mem_info.rss / (1024 ** 2))  # Convert bytes to MB

# Function to get current GPU memory usage
def get_gpu_memory_usage():
    result = subprocess.run(
        ["nvidia-smi", "--query-gpu=memory.used", "--format=csv,nounits,noheader"],
        stdout=subprocess.PIPE,
        text=True,
        check=True
    )
    gpu_memory = result.stdout.strip()
    return int(gpu_memory)  # Memory in MB

### DistilBERT

In [4]:
!nvidia-smi

Tue Jul 16 20:06:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off | 00000000:01:00.0 Off |                    0 |
| N/A   38C    P0              66W / 500W |    806MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [5]:
# GPU Meomory Usage
initial_gpu_memory = get_gpu_memory_usage()
print(f"Initial GPU memory usage of distilBERT: {initial_gpu_memory} MB")

# Load the model
pipe_distilbert = pipeline("text-classification", model="AdamCodd/distilbert-base-uncased-finetuned-sentiment-amazon", device=0)

# Memory Usage Calculation
after_loading_gpu_memory = get_gpu_memory_usage()
print(f"GPU memory usage after loading distilBERT: {after_loading_gpu_memory} MB")

gpu_memory_used_by_model = after_loading_gpu_memory - initial_gpu_memory

print(f"GPU memory used by distilBERT: {gpu_memory_used_by_model} MB")

Initial GPU memory usage of distilBERT: 806 MB
GPU memory usage after loading distilBERT: 1518 MB
GPU memory used by distilBERT: 712 MB


In [5]:
# CPU Meomory Usage
initial_cpu_memory = get_cpu_memory_usage()
print(f"Initial CPU memory usage of distilBERT: {initial_cpu_memory} MB")

# Load the model
pipe_distilbert = pipeline("text-classification", model="AdamCodd/distilbert-base-uncased-finetuned-sentiment-amazon", device=-1)

# Memory Usage Calculation
after_loading_cpu_memory = get_cpu_memory_usage()
print(f"CPU memory usage after loading distilBERT: {after_loading_cpu_memory} MB")

cpu_memory_used_by_model = after_loading_cpu_memory - initial_cpu_memory

print(f"CPU memory used by distilBERT: {cpu_memory_used_by_model} MB")

Initial CPU memory usage of distilBERT: 505 MB
CPU memory usage after loading distilBERT: 777 MB
CPU memory used by distilBERT: 272 MB


### ELECTRA

In [3]:
!nvidia-smi

Tue Jul 16 20:09:09 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off | 00000000:01:00.0 Off |                    0 |
| N/A   38C    P0              66W / 500W |    806MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [4]:
# GPU Meomory Usage
initial_gpu_memory = get_gpu_memory_usage()
print(f"Initial GPU memory usage of ELECTRA: {initial_gpu_memory} MB")

# Load the model
pipe_electra = pipeline("text-classification", model="pig4431/amazonPolarity_ELECTRA_5E", device=0)

# Memory Usage Calculation
after_loading_gpu_memory = get_gpu_memory_usage()
print(f"GPU memory usage after loading ELECTRA: {after_loading_gpu_memory} MB")

gpu_memory_used_by_model = after_loading_gpu_memory - initial_gpu_memory

print(f"GPU memory used by ELECTRA: {gpu_memory_used_by_model} MB")

Initial GPU memory usage of ELECTRA: 806 MB
GPU memory usage after loading ELECTRA: 1698 MB
GPU memory used by ELECTRA: 892 MB


In [3]:
# CPU Meomory Usage
initial_cpu_memory = get_cpu_memory_usage()
print(f"Initial CPU memory usage of ELECTRA: {initial_cpu_memory} MB")

# Load the model
pipe_electra = pipeline("text-classification", model="pig4431/amazonPolarity_ELECTRA_5E", device=-1)

# Memory Usage Calculation
after_loading_cpu_memory = get_cpu_memory_usage()
print(f"CPU memory usage after loading ELECTRA: {after_loading_cpu_memory} MB")

cpu_memory_used_by_model = after_loading_cpu_memory - initial_cpu_memory

print(f"CPU memory used by ELECTRA: {cpu_memory_used_by_model} MB")

Initial CPU memory usage of ELECTRA: 494 MB
CPU memory usage after loading ELECTRA: 930 MB
CPU memory used by ELECTRA: 436 MB


### Flan-T5

In [4]:
!nvidia-smi

Tue Jul 16 20:10:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off | 00000000:01:00.0 Off |                    0 |
| N/A   38C    P0              66W / 500W |    806MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [5]:
# GPU Meomory Usage
initial_gpu_memory = get_gpu_memory_usage()
print(f"Initial GPU memory usage of Flan-T5: {initial_gpu_memory} MB")

# Load the model
pipeFlanT5 = pipeline("text2text-generation", model="google/flan-t5-xxl", device=0)

# Memory Usage Calculation
after_loading_gpu_memory = get_gpu_memory_usage()
print(f"GPU memory usage after loading Flan-T5: {after_loading_gpu_memory} MB")

gpu_memory_used_by_model = after_loading_gpu_memory - initial_gpu_memory

print(f"GPU memory used by Flan-T5: {gpu_memory_used_by_model} MB")

Initial GPU memory usage of Flan-T5: 806 MB


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

GPU memory usage after loading Flan-T5: 43704 MB
GPU memory used by Flan-T5: 42898 MB


In [6]:
# CPU Meomory Usage
initial_cpu_memory = get_cpu_memory_usage()
print(f"Initial CPU memory usage of Flan-T5: {initial_cpu_memory} MB")

# Load the model
pipeFlanT5 = pipeline("text2text-generation", model="google/flan-t5-xxl", device=-1)

# Memory Usage Calculation
after_loading_cpu_memory = get_cpu_memory_usage()
print(f"CPU memory usage after loading Flan-T5: {after_loading_cpu_memory} MB")

cpu_memory_used_by_model = after_loading_cpu_memory - initial_cpu_memory

print(f"CPU memory used by Flan-T5: {cpu_memory_used_by_model} MB")

Initial CPU memory usage of Flan-T5: 1099 MB


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

CPU memory usage after loading Flan-T5: 43614 MB
CPU memory used by Flan-T5: 42515 MB


### Flan-UL2

In [3]:
!nvidia-smi

Tue Jul 16 20:12:52 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off | 00000000:01:00.0 Off |                    0 |
| N/A   38C    P0              66W / 500W |    806MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [4]:
# GPU Meomory Usage
initial_gpu_memory = get_gpu_memory_usage()
print(f"Initial GPU memory usage of Flan-UL2: {initial_gpu_memory} MB")

# Load the model
pipeFlanUL2 = pipeline("text2text-generation", model="google/flan-ul2", device=0)

# Memory Usage Calculation
after_loading_gpu_memory = get_gpu_memory_usage()
print(f"GPU memory usage after loading Flan-UL2: {after_loading_gpu_memory} MB")

gpu_memory_used_by_model = after_loading_gpu_memory - initial_gpu_memory

print(f"GPU memory used by Flan-UL2: {gpu_memory_used_by_model} MB")

Initial GPU memory usage of Flan-UL2: 806 MB


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

GPU memory usage after loading Flan-UL2: 75460 MB
GPU memory used by Flan-UL2: 74654 MB


In [5]:
# CPU Meomory Usage
initial_cpu_memory = get_cpu_memory_usage()
print(f"Initial CPU memory usage of Flan-UL2: {initial_cpu_memory} MB")

# Load the model
pipeFlanUL2 = pipeline("text2text-generation", model="google/flan-ul2", device=-1)

# Memory Usage Calculation
after_loading_cpu_memory = get_cpu_memory_usage()
print(f"CPU memory usage after loading Flan-UL2: {after_loading_cpu_memory} MB")

cpu_memory_used_by_model = after_loading_cpu_memory - initial_cpu_memory

print(f"CPU memory used by Flan-UL2: {cpu_memory_used_by_model} MB")

Initial CPU memory usage of Flan-UL2: 674 MB


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

CPU memory usage after loading Flan-UL2: 74945 MB
CPU memory used by Flan-UL2: 74271 MB


|Model|DistilBERT|ELECTRA|FlanT5|FlanUL2|
|-|-|-|-|-|
|GPU(MB)|712|892|42898|74654|
|CPU(MB)|272|436|42515|74271|

### FLOPs Calculation

### DistilBERT

In [6]:
# Load the model
model_name = "AdamCodd/distilbert-base-uncased-finetuned-sentiment-amazon"
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define an example input
example_text = "This is a great product!"
inputs = tokenizer(example_text, return_tensors="pt")
example_input = (inputs['input_ids'],)

# Calculate MACs and params
macs, params = profile(model, inputs=example_input)
flops = macs * 2  # Each MAC operation corresponds to 2 FLOPs (1 multiplication + 1 addition)
macs, flops, params = clever_format([macs, flops, params], "%.3f")

print(f"MACs: {macs}")
print(f"FLOPs: {flops}")
print(f"Number of parameters: {params}")

[INFO] Register count_normalization() for <class 'torch.nn.modules.normalization.LayerNorm'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
MACs: 340.649M
FLOPs: 681.299M
Number of parameters: 43.121M


### ELECTRA

In [7]:
# Load the model
model_name = "pig4431/amazonPolarity_ELECTRA_5E"
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define an example input
example_text = "This is a great product!"
inputs = tokenizer(example_text, return_tensors="pt")
example_input = (inputs['input_ids'],)

# Calculate MACs and params
macs, params = profile(model, inputs=example_input)
flops = macs * 2  # Each MAC operation corresponds to 2 FLOPs (1 multiplication + 1 addition)
macs, flops, params = clever_format([macs, flops, params], "%.3f")

print(f"MACs: {macs}")
print(f"FLOPs: {flops}")
print(f"Number of parameters: {params}")

[INFO] Register count_normalization() for <class 'torch.nn.modules.normalization.LayerNorm'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
MACs: 680.683M
FLOPs: 1.361G
Number of parameters: 85.648M


### Flan-T5

In [8]:
# Load the model
model_name = "google/flan-t5-xxl"
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define an example input
example_text = "This is a great product!"
inputs = tokenizer(example_text, return_tensors="pt")
example_input = (inputs['input_ids'],)

# Calculate MACs and params
macs, params = profile(model, inputs=example_input)
flops = macs * 2  # Each MAC operation corresponds to 2 FLOPs (1 multiplication + 1 addition)
macs, flops, params = clever_format([macs, flops, params], "%.3f")

print(f"MACs: {macs}")
print(f"FLOPs: {flops}")
print(f"Number of parameters: {params}")

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at google/flan-t5-xxl and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
MACs: 86.990G
FLOPs: 173.980G
Number of parameters: 10.888G


### Flan-UL2

In [9]:
# Load the model
model_name = "google/flan-ul2"
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define an example input
example_text = "This is a great product!"
inputs = tokenizer(example_text, return_tensors="pt")
example_input = (inputs['input_ids'],)

# Calculate MACs and params
macs, params = profile(model, inputs=example_input)
flops = macs * 2  # Each MAC operation corresponds to 2 FLOPs (1 multiplication + 1 addition)
macs, flops, params = clever_format([macs, flops, params], "%.3f")

print(f"MACs: {macs}")
print(f"FLOPs: {flops}")
print(f"Number of parameters: {params}")

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at google/flan-ul2 and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
MACs: 154.636G
FLOPs: 309.271G
Number of parameters: 19.344G


|Model|DistilBERT|ELECTRA|FlanT5|FlanUL2|
|-|-|-|-|-|
|MACs|340.649M|680.683M|86.990G|154.636G|
|FLOPs|681.299M|1.361G|173.980G|309.271G|
|Params|43.121M|85.648M|10.888G|19.344G|